Select command based on score

In [5]:
from datasets import load_from_disk

ins1_scores = load_from_disk("data/scoring/ins1_score")
ins2_scores = load_from_disk("data/scoring/ins2_score")
ins1_scores, ins2_scores

(Dataset({
     features: ['quality', 'difficulty', 'relevance_to_medicine', 'mention_specific_details', 'ids'],
     num_rows: 209971
 }),
 Dataset({
     features: ['quality', 'difficulty', 'relevance_to_medicine', 'mention_specific_details', 'ids'],
     num_rows: 209971
 }))

In [6]:
import numpy as np

# Loading .npy files
successful_indexes1 = np.load('data/scoring/successful_indexes1.npy')
successful_indexes2 = np.load('data/scoring/successful_indexes2.npy')

# Convert the two lists to sets and then take the intersection
common_elements = list(set(successful_indexes1) & set(successful_indexes2))
len(common_elements)

209971

In [ ]:
# Use the filter method to filter out the items whose ids are in ids_list
ins1_scores = ins1_scores.filter(lambda x: x['ids'] in common_elements)
ins2_scores = ins2_scores.filter(lambda x: x['ids'] in common_elements)
ins1_scores, ins2_scores

In [7]:
ins1 = load_from_disk("data/instructions/gen_ins/ins1")
ins1 = ins1.select(common_elements)
ins1 = ins1["question"]
ins2 = load_from_disk("data/instructions/gen_ins/ins2")
ins2 = ins2.select(common_elements)
ins2 = ins2["question"]
ins1[0], ins2[0], len(ins1), len(ins2)

('What factors contribute to the low efficiency of cationic lipid-mediated gene transduction in polarized and differentiated airway epithelial cells, and how might these barriers be overcome to improve gene therapy for airway diseases such as cystic fibrosis?',
 'How do the results of gene transduction using GL-67:pDNA complexes in the nasal epithelium of cystic fibrosis transgenic mice compare to those observed in the lungs, and what implications do these differences have for the development of gene therapy strategies for cystic fibrosis?',
 209971,
 209971)

In [8]:
import numpy as np

# 加载 .npy 文件
successful_indexes = np.load('data/instructions/gen_ins/successful_indexes.npy')

# 打印加载的数据
print(successful_indexes)

[     0      1      2 ... 209997 209998 209999]


In [ ]:
dataset = load_from_disk("raw_medical_text")
# 若存在没有 json 解析成功的
dataset = dataset.select(successful_indexes)
dataset = dataset.select(common_elements)

texts = dataset["text"]
dataset, texts[0], len(texts)

In [12]:
ins1_scores[1], ins2_scores[1]

({'quality': 8,
  'difficulty': 7,
  'relevance_to_medicine': 4,
  'mention_specific_details': 'False',
  'ids': 1},
 {'quality': 8,
  'difficulty': 6,
  'relevance_to_medicine': 4,
  'mention_specific_details': 'False',
  'ids': 1})

Use a list to store the selected question numbers (1/2), and use 0 to indicate that neither question is selected.

In [13]:
def IsMentionSpecificDetails(ins1_score, ins2_score):
    if ins1_score["mention_specific_details"] == "True" and ins2_score["mention_specific_details"] == "True" :
        return 0
    elif ins1_score["mention_specific_details"] == "True" :
        return 2
    elif ins2_score["mention_specific_details"] == "True" :
        return 1
    else:
        return -1

Add one to remove none and use it according to the situation

In [14]:
def IsExistNone(ins1_score, ins2_score):
    if (ins1_score["quality"] == None or ins1_score["difficulty"] == None or ins1_score["relevance_to_medicine"] == None) and (ins2_score["quality"] == None or ins2_score["difficulty"] == None or ins2_score["relevance_to_medicine"] == None):
        return 0
    elif ins1_score["quality"] == None or ins1_score["difficulty"] == None or ins1_score["relevance_to_medicine"] == None:
        return 2
    elif ins2_score["quality"] == None or ins2_score["difficulty"] == None or ins2_score["relevance_to_medicine"] == None:
        return 1
    else:
        return -1

In [15]:
def WhoIsBetter(ins1_score, ins2_score):
    # Determine whether specific details are mentioned
    flag = IsMentionSpecificDetails(ins1_score, ins2_score)
    # print(f"flag: {flag}")
    if flag != -1:
        return flag
    flag = IsExistNone(ins1_score, ins2_score)
    if flag != -1:
        print(f"There is a model output error, flag = {flag}")
        return flag
    # Compare total scores
    ins1_score_sum = ins1_score["quality"] + ins1_score["difficulty"] + ins1_score["relevance_to_medicine"]
    ins2_score_sum = ins2_score["quality"] + ins2_score["difficulty"] + ins2_score["relevance_to_medicine"]
    if ins1_score_sum > ins2_score_sum:
        return 1
    elif ins1_score_sum < ins2_score_sum:
        return 2
    # Compare quality + difficulty
    ins1_score_sum = ins1_score["quality"] + ins1_score["difficulty"]
    ins2_score_sum = ins2_score["quality"] + ins2_score["difficulty"]
    if ins1_score_sum > ins2_score_sum:
        return 1
    elif ins1_score_sum < ins2_score_sum:
        return 2
    # Compare quality
    ins1_score_sum = ins1_score["quality"]
    ins2_score_sum = ins2_score["quality"]
    if ins1_score_sum > ins2_score_sum:
        return 1
    elif ins1_score_sum < ins2_score_sum:
        return 2
    else:
        # Randomly select 1 or 2
        return random.choice([1, 2])

In [16]:
import random
from tqdm import tqdm

ins_num_list = []
for ins1_score, ins2_score in tqdm(zip(ins1_scores, ins2_scores)):
    better = WhoIsBetter(ins1_score, ins2_score)
    # print(ins1_score)
    # print(ins2_score)
    # print(better)
    if better == 1:
        ins_num_list.append(1)
    elif better == 2:
        ins_num_list.append(2)
    else:
        ins_num_list.append(0)
print(ins_num_list)

209971it [00:14, 14085.96it/s]

[0, 1, 0, 2, 0, 2, 1, 2, 1, 2, 2, 1, 1, 1, 0, 1, 1, 0, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 0, 2, 0, 1, 2, 1, 2, 1, 1, 1, 1, 2, 0, 1, 2, 1, 2, 0, 1, 0, 1, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 0, 2, 2, 2, 1, 2, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 0, 1, 2, 1, 0, 1, 0, 0, 0, 1, 0, 0, 2, 0, 1, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 0, 1, 2, 2, 1, 1, 2, 1, 1, 1, 2, 1, 2, 2, 0, 1, 2, 1, 1, 2, 1, 0, 1, 2, 2, 2, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 0, 2, 2, 2, 1, 1, 0, 1, 2, 0, 0, 0, 2, 1, 2, 0, 1, 1, 0, 0, 1, 2, 2, 2, 0, 2, 1, 2, 0, 0, 0, 2, 1, 2, 1, 1, 1, 2, 0, 1, 1, 0, 1, 1, 1, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 0, 0, 2, 0, 0, 1, 2, 2, 1, 2, 2, 1, 1, 1, 0, 0, 2, 1, 2, 0, 2, 2, 0, 1, 2, 2, 1, 0, 2, 0, 1, 2, 2, 1, 1, 1, 0, 2, 2, 2, 1, 0, 2, 1, 0, 2, 2, 1, 2, 2, 2, 1, 2, 1, 1, 1, 0, 1, 1, 0, 1, 0, 2, 2, 1, 2, 2, 1, 1, 0, 1, 0, 1, 0, 1, 2, 0, 0, 1, 2, 1, 1, 1, 2, 0, 1, 2, 0, 0, 0, 1, 0, 2, 2, 1, 0, 2, 1, 2, 1, 1, 0, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 0, 2, 1, 0, 0, 1, 2, 0, 2, 1, 0, 1, 2, 2, 0, 1, 2, 

In [17]:
count = 0
for value in ins_num_list:
    if value == 0:
        count += 1
len(ins_num_list) - count

169701

In [18]:
select_text_list, select_ins_list, select_ids_list, select_complex_list, select_quality_list, select_language_list = [], [], [], [], [], []
for ids, value in enumerate(tqdm(ins_num_list)):
    if value == 1:
        select_text_list.append(texts[ids])
        select_ins_list.append(ins1[ids])
        select_ids_list.append(ids)
        select_complex_list.append(ins1_scores[ids]["difficulty"])
        select_quality_list.append(ins1_scores[ids]["quality"])
        select_language_list.append(dataset[ids]["language"])
    elif value == 2:
        select_text_list.append(texts[ids])
        select_ins_list.append(ins2[ids])
        select_ids_list.append(ids)
        select_complex_list.append(ins2_scores[ids]["difficulty"])
        select_quality_list.append(ins2_scores[ids]["quality"])
        select_language_list.append(dataset[ids]["language"])
select_text_list[0], select_ins_list[0], select_ids_list[0], select_complex_list[0], select_language_list[0], len(select_text_list), len(select_ins_list), len(select_ids_list), len(select_complex_list), len(select_language_list)

100%|██████████| 209971/209971 [00:31<00:00, 6761.10it/s]


('There are some components that must be thought of for a business to run smoothly and for targets to be met totally. Consequently, the state refuses to reimburse such clinics for many medical companies rendered to their Medicaid patients, even long-established ones. My current health provider presents the most effective care I bear in mind receiving in my many adult years. The follow tries to help patients falling between the cracks of our increasingly inefficient health care system.\\n\\nThe survival of ladies in childbirth reflects the overall growth of a country and whether or not the health companies are functioning. Intermediate causes, which are the first and second delays in care-in search of, embody the low social status of ladies, lack of knowledge and data at the household stage, inadequate sources to seek care, and poor access to quality health care.\\n\\nMoney penalties are enforced for non-compliance by healthcare entities. The notice of privacy practices should be in wri

In [20]:
from datasets import Dataset

ins_text_dict = {"ids": select_ids_list, "text": select_text_list, 
"instruction": select_ins_list, "complexity": select_complex_list,
"quality": select_quality_list, "language": select_language_list}
ins_text_set = Dataset.from_dict(ins_text_dict)
ins_text_set

Dataset({
    features: ['ids', 'text', 'instruction', 'complexity', 'quality', 'language'],
    num_rows: 169701
})

In [21]:
ins_text_set[4]

{'ids': 7,
 'text': 'A COUPLE have been awarded #100,000 compensation after their son was severely brain damaged at birth by a medical error. Mary Murtagh reports\nIT WAS when a doctor told Lyndsay Carney that her son would never walk or talk that she fully realised how badly injured he had been during his birth.\nLittle Michael Tierney was left blind, with cerebral palsy and epilepsy after being starved of oxygen.\nA catalogue of errors by medical staff as he was born left Michael needing round the clock care.\nLiverpool Women\'s hospital admitted responsibility and apologised the day after the two-year-old died. Today it said it had learned important lessons from the tragedy. The family have been awarded #100,000 in compensation.\nThere was more heartache for the family; two days after Michael\'s death his carer Craig Eaton, 22, was shot dead.\nHe was walking home after a night out drowning his sorrows over his young charge\'s death - his killers have never been caught, despite a #10

In [22]:
ins_text_set.save_to_disk("data/scoring/ins_text")

Saving the dataset (2/2 shards): 100%|██████████| 169701/169701 [00:00<00:00, 176794.83 examples/s]
